In [1]:
import numpy as np
import pandas as pd
import io
import re
from collections import Counter
import gc

import pickle
import nltk
import math
from sklearn.model_selection import train_test_split

In [2]:
with open('preprocessed_CETEN.pkl', 'rb') as input:
    phrases, _, _, _ = pickle.load(input)

#### Divide a base em treino e teste

In [3]:
phrases_train , phrases_test = train_test_split(phrases,test_size=0.2, random_state=42)

#### Converte qualquer palavra com freq < 5 para __RARE__

In [4]:
min_threshold = 5

word_counts = Counter()
for phrase in phrases_train:
    for word in phrase:
        word_counts[word[0]]+=1
        
word_counts = {word: count for word, count in word_counts.items() if word_counts[word] >= min_threshold}

In [5]:
phrases_train_rare = []
RARE_WORD = '__RARE__'
for phrase in phrases_train:
    phrases_train_rare.append([(w[0] if word_counts.get(w[0]) else RARE_WORD,w[1]) for w in phrase])

#### Contagem de palavras na base de treino

In [6]:
tags_counter = Counter()
for s in phrases_train_rare:
    for tk in s:
        tag = tk[1]
        if tags_counter[tag]:
            tags_counter[tag]=tags_counter[tag]+1
        else:
            tags_counter[tag] = 1

In [7]:
allowed_tags = tags_counter.keys()

#### Contagem de etiquetas na base de treino

In [8]:
word_freq = {}
for s in phrases_train_rare:
    for tk in s:
        if word_freq.get(tk[0]) == None:
            word_freq[tk[0]] = Counter()
        word_freq[tk[0]][tk[1]] = word_freq[tk[0]][tk[1]] + 1

In [9]:
def get_emission_probs(word_freq, tags_counter):
    smooth_emission_not_seen = 0
    e = {}
    for word in word_freq.keys():
        for tag in tags_counter.keys():
            if tags_counter[tag] == 0:
                print(tag)
            e[(word,tag)] = float((word_freq[word].get(tag,smooth_emission_not_seen)+1)/float(tags_counter[tag]))
    return e
e_prob = get_emission_probs(word_freq, tags_counter)    

In [10]:
smooth_transition_not_seen = 0

def prep_sentence(phrase):
    return ['*'] + ['*'] + [word[1] for word in phrase] + ['STOP']
    
def get_transition_probs(phrases,word_feq,tags_counter):

    bigrams_c = Counter()
    for phrase in phrases:
        bigrams = nltk.bigrams(prep_sentence(phrase))
        for bigram in bigrams:
            bigrams_c[bigram] +=1

    trigrams_c = Counter()
    for phrase in phrases:
        trigrams = nltk.trigrams(prep_sentence(phrase))
        for trigram in trigrams:
            trigrams_c[trigram] +=1

    trigrams_p = {}
    for trigram, trigram_count in trigrams_c.items():
        bigram = (trigram[0],trigram[1])
        bigram_count = bigrams_c.get(bigram)
        if bigram_count:
            trigrams_p[trigram] = float(trigram_count)/float(bigram_count)
        else:
            trigrams_p[trigram] = 0
        
    return trigrams_p, bigrams_c
    
    
q_prob, bigrams_c = get_transition_probs(phrases,word_freq,tags_counter)

In [11]:
import timeit

In [39]:
start_time = timeit.default_timer()
c = 0
for j in range(100):
    for i in range(1764):
        c=c+1
elapsed = timeit.default_timer() - start_time

elapsed

0.027470087256347142

In [ ]:
e_pro

In [76]:
start_time = timeit.default_timer()
c = 0
for w in word_freq.keys():
    for k in allowed_tags:
        e_prob[(w,k)]
        #c=c+1
    c=c+1
elapsed = timeit.default_timer() - start_time

elapsed

1.227884536436477

In [53]:
e_prob[('o','DET')]

0.35558957002803915

In [72]:
len(word_freq.keys())

88104

In [71]:
len(q_prob.keys())

9710

In [99]:
def viterbi(sents, e_prob,q_prob,allowed_tags, tdef, tloop1, tloop2, ttag):
    
    start_time = timeit.default_timer()

    
    def S(k):
        if k == -1 or k == 0:
            return ['*']
        else:
            return allowed_tags
    pi = {}
    pi[(0, '*','*')] = 1

    bp = {}
    
    elapsed = timeit.default_timer() - start_time
    tdef=tdef+elapsed
    #print("Time definition: ", elapsed)

    tagged = []
    for sent in sents:
        c=0
        start_time = timeit.default_timer()
        for k in range(1, len(sent)+1):
            for u in S(k-1):
                for v in S(k):
                    max_p = -1
                    max_tag = None
                    for w in S(k-2):
                        c=c+1
                        #if bigrams_c[(w,u)] > 0: ### Se o bigram não existir na base de treino, não faz sentido testá-lo aqui 
                        prob = pi[(k-1,w,u)] * q_prob.get((w,u,v),0) * e_prob[(sent[k-1],v)]
                        #prob = 0
                        if (prob > max_p):
                            max_p = prob
                            max_tag = w
                    pi[(k,u,v)] = max_p
                    bp[(k,u,v)] = max_tag

        elapsed = timeit.default_timer() - start_time
        tloop1+=elapsed
        #print("First loop: ", elapsed)
        print(c)

        start_time = timeit.default_timer()
        max_p = -1
        max_u_tag = None
        max_v_tag = None
        n = len(sent)
        for u in S(n-1):
            for v in S(n):
                #if bigrams_c[(u,v)] > 0:
                prob = pi[(n,u,v)] * q_prob.get((u,v,'STOP'),0)
                if (prob > max_p):
                    max_p = prob
                    max_u_tag = u
                    max_v_tag = v

        elapsed = timeit.default_timer() - start_time
        tloop2+=elapsed
        #print("Second loop: ", elapsed)

        start_time = timeit.default_timer()
        tags = []
        tags.append(max_v_tag)
        tags.append(max_u_tag)
        for i,k in enumerate(range(n-2,0, -1)):
            tags.append(bp[(k+2, tags[i+1], tags[i])])

        tags = list(reversed(tags))

        tagged_sentence = []
        for j in range(0, n):
            tagged_sentence.append((sent[j], tags[j]))

        elapsed = timeit.default_timer() - start_time
        ttag+=elapsed
        #print("Tags: ", elapsed)
        
        tagged.append(tagged_sentence)
    
    return tagged, tdef, tloop1, tloop2, ttag

In [100]:
tdef=0
tloop1=0
tloop2=0
ttag=0
corretas_baseline = 0
corretas_viterbi = 0
totais = 0

testwith = 10
for s in phrases_test[:testwith]:
    sents_with_rare = []
    sents_with_rare.append([tk[0] if word_counts.get(tk[0]) != None else RARE_WORD for tk in s])
    preds,tdef, tloop1, tloop2, ttag = viterbi(sents_with_rare, e_prob, q_prob, allowed_tags, tdef, tloop1, tloop2, ttag)
    for tk_golden, tk_pred in zip(s,preds[0]):
        totais+=1
        if tk_golden[1] == word_freq[tk_pred[0]].most_common(1)[0][0]:
            corretas_baseline+=1
        if tk_golden[1] == tk_pred[1]:
            corretas_viterbi+=1

372246
742686
1409478
42
890862
816774
1854006
2594886
2298534
1113126


In [98]:
tdef, tloop1, tloop2, ttag

(1.5394893125630915e-05,
 3.6145911862834055,
 0.010518910526343461,
 0.0002663783000116382)

In [26]:
corretas_viterbi, corretas_baseline, totais, corretas_viterbi/totais*100, corretas_baseline/totais*100

(319, 350, 367, 86.92098092643052, 95.36784741144415)

In [358]:
len(sents_with_rare)

1000

In [2]:
from multiprocessing import Pool

In [3]:
def process_sent(s):
    return 2**s

In [4]:
pool = Pool(2) 

In [ ]:
pool.map(process_sent, [1,2,3,4])

In [ ]:
from multiprocessing import Pool

pool = Pool(2) 

def process_sent(s):
    return 2**s
    

pool.map(process_sent, [1,2,3,4])

In [ ]:
with Pool() as pool: 
    pool(fits.open, sents_with_rare)

In [ ]:
from multiprocessing import Pool

def process_sent(s):
    preds,_, _,_,_ = viterbi([s], e_prob, q_prob, allowed_tags, 0, 0, 0, 0)
    return preds[0]
    
pool = Pool(2) 
preds = pool.map(process_sent, sents_with_rare[:10])

In [356]:
tdef=0
tloop1=0
tloop2=0
ttag=0
corretas_baseline = 0
corretas_viterbi = 0
totais = 0
sents_with_rare = []

testwith = 1000
for s in phrases_test[:testwith]:
    sents_with_rare.append([tk[0] if word_counts.get(tk[0]) != None else RARE_WORD for tk in s])

#preds,tdef, tloop1, tloop2, ttag = viterbi(sents_with_rare, e_prob, q_prob, allowed_tags, tdef, tloop1, tloop2, ttag)
#for s_golden, s_pred in zip(phrases_test[:testwith],preds):
#    for tk_golden, tk_pred in zip(s_golden,s_pred):
#        totais+=1
#        if tk_golden[1] == word_freq[tk_pred[0]].most_common(1)[0][0]:
#            corretas_baseline+=1
#        if tk_golden[1] == tk_pred[1]:
#            corretas_viterbi+=1

In [345]:
corretas_viterbi, corretas_baseline, totais, corretas_viterbi/totais*100, corretas_baseline/totais*100

(42, 173, 182, 23.076923076923077, 95.05494505494505)

In [348]:
tdef, tloop1, tloop2, ttag

(1.8660475689102896e-06,
 900.5387265531094,
 1.0598030946530343,
 0.032637173161219835)

In [336]:
corretas_viterbi, corretas_baseline, totais, corretas_viterbi/totais*100, corretas_baseline/totais*100

(501, 504, 530, 94.52830188679245, 95.09433962264151)

In [320]:
tdef, tloop1, tloop2, ttag

(4.245258369905969e-05,
 25.797425600680697,
 0.02671713703517753,
 0.0009236935800345236)

In [303]:
corretas_viterbi, corretas_baseline, totais, corretas_viterbi/totais*100, corretas_baseline/totais*100

(501, 504, 530, 94.52830188679245, 95.09433962264151)

In [292]:
#phrase = 'ele caiu de a escada e morreu em a hora'.split(' ')
phrase = [tk[0] if word_counts.get(tk[0]) != None else RARE_WORD for tk in phrases_test[0]]
viterbi(phrase, e_prob, q_prob, allowed_tags)

TypeError: viterbi() missing 1 required positional argument: 'e_values'

In [290]:
from collections import defaultdict, deque

START_SYMBOL = '*'
STOP_SYMBOL = 'STOP'
RARE_SYMBOL = '__RARE__'

def viterbi(brown_dev_words, taglist, known_words, q_values, e_values):
    tagged = []

    # pi[(k, u, v)]: max probability of a tag sequence ending in tags u, v at position k
    # bp[(k, u, v)]: backpointers to recover the argmax of pi[(k, u, v)]
    pi = defaultdict(float)
    bp = {}

    # Initialization
    pi[(0, START_SYMBOL, START_SYMBOL)] = 1

    # Define tagsets S(k)
    def S(k):
        if k in (-1, 0):
            return {START_SYMBOL}
        else:
            return taglist

    # The Viterbi algorithm
    for sent_words_actual in brown_dev_words:
        sent_words = [word if word in known_words else RARE_SYMBOL for word in sent_words_actual]
        n = len(sent_words)
        for k in range(1, n+1):
            for u in S(k-1):
                for v in S(k):
                    max_score = -1
                    max_tag = None
                    for w in S(k - 2):
                        #if e_values.get((sent_words[k-1], v), 0) != 0:
                        score = pi[(k-1, w, u)] * \
                                q_values.get((w, u, v),0) * \
                                e_values[(sent_words[k-1], v)]
                        if score > max_score:
                            max_score = score
                            max_tag = w
                    pi[(k, u, v)] = max_score
                    bp[(k, u, v)] = max_tag

        max_score = -1
        u_max, v_max = None, None
        for u in S(n-1):
            for v in S(n):
                score = pi[(n, u, v)] * \
                        q_values.get((u, v, STOP_SYMBOL), 0)
                if score > max_score:
                    max_score = score
                    u_max = u
                    v_max = v

        tags = []
        tags.append(v_max)
        tags.append(u_max)

        for i, k in enumerate(range(n-2, 0, -1)):
            tags.append(bp[(k+2, tags[i+1], tags[i])])
        tags = list(reversed(tags))

        tagged_sentence = []
        for j in range(0, n):
            tagged_sentence.append((sent_words_actual[j] , tags[j]))
        tagged.append(tagged_sentence)
        

    return tagged

In [299]:
phr = [list(map(lambda x: x[0], ph)) for ph in phrases_test[:30]]
phrases_viterbi = viterbi(phr, allowed_tags, set(word_counts.keys()), q_prob, e_prob)

corretas = 0
totais = 0
for s_viterbi, s_gold in zip(phrases_viterbi, phrases_test[:30]):
    for tk_viterbi, tk_gold in zip(s_viterbi, s_gold):
        totais+=1
        if tk_gold[1] == tk_viterbi[1]:
            corretas+=1
corretas,totais

(501, 530)